<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=05dcbd7e846ec5d0ac292d28eba63fbf82d2bad338f7a802150d3dd69b7b75f8
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-22 12:46:44
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  66.76 K
Current:  1.41 C
-------------------
Today PnL: 96.51 K (0.69%)
Current PnL: -25.86 L (-16.71%)
CY Booked + Current PnL: -11.57 L (-7.48%)
-------------------
Total profit:  1.60 L
Total loss:  -27.47 L
-------------------
Total Booked + Current PnL: 15.41 L (12.05%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.12%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.32 C
Est FTT PnL: 90.50 L (64.1%)
Deployed:  1.28 C
Current:  1.41 C
CAGR/XIRR %: 5.49%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,0.02,-19.62,24.52,0.09,19862.0,-19773.0,81004.0,88.05,32.0,M-SC,2.27,253.0,-1.00,0.58,6.42,OX40N,NTT,DURABLES
82,VOLTAS,1530.00,0.69,8.35,10.47,19.69,21751.0,16008.0,207750.0,-2.18,54.0,X-MC,2.46,78.0,0.74,1.48,15.71,XY25,NTT,AC
50,MASFIN,398.61,-1.31,-4.53,27.84,22.05,26042.0,-4440.0,93540.0,-17.80,48.0,H-SC,6.42,164.0,-0.17,0.67,35.97,XR,ATH,FINANCE
39,INDIGOPNTS,1408.00,-0.36,-15.87,18.89,0.02,27741.0,-27706.0,146853.0,123.77,43.0,M-SC,12.62,234.0,-1.00,1.05,27.74,OX40N,NTT,PAINTS
43,ITC,452.00,0.54,-1.18,12.10,10.78,28736.0,-2833.0,237485.0,-41.15,50.0,X-LC,2.15,5.0,-0.10,1.69,4.00,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RAJESHEXPO,518.00,11.17,-56.11,128.01,0.07,77066.0,-76974.0,60203.0,2107.07,77.0,L-SC,3.07,268.0,-1.00,0.43,49.73,OX40N,NTT,JEWELLERY
28,GREENPANEL,537.00,3.11,-37.56,129.24,43.14,147739.0,-68764.0,114314.0,155.81,39.0,M-SC,10.08,240.0,-0.47,0.81,7.53,XY24,NTT,MISC
81,VBL,671.64,3.06,-2.28,38.84,35.67,119873.0,-7210.0,308632.0,-13.89,59.0,X-LC,3.78,4.0,-0.06,2.20,11.26,X40N,ATH,FMCG
13,BSOFT,831.70,2.99,-19.07,86.31,50.79,98248.0,-26821.0,113832.0,3.98,71.0,H-SC,7.22,151.0,-0.27,0.81,32.54,XR,ATH,IT
84,WIPRO,420.00,2.91,11.62,54.33,72.27,102762.0,19696.0,189144.0,-4.26,77.0,M-LC,3.37,99.0,0.19,1.35,18.52,XR,NTT,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COFFEEDAY,80.00,-2.92,-41.55,131.41,35.27,87222.0,-47175.0,66374.0,-55.73,47.0,L-SC,10.07,270.0,-0.54,0.47,62.45,XR,NTT,HOTELS
44,JCHAC,2282.00,-1.98,-36.35,57.09,-0.01,46443.0,-46454.0,81351.0,16552.17,34.0,M-SC,10.47,233.0,-1.00,0.58,4.12,OX40N,NTT,AC
8,AWL,485.00,-1.65,-23.79,101.41,53.50,233206.0,-71787.0,229964.0,-63.12,34.0,X-MC,4.82,58.0,-0.31,1.64,2.14,XY24,NTT,FMCG
57,RAJOOENG,143.10,-1.35,-37.34,122.79,39.61,78868.0,-38270.0,64230.0,-57.57,33.0,H-SC,30.05,135.0,-0.49,0.46,1.20,AR,ATH,MISC
50,MASFIN,398.61,-1.31,-4.53,27.84,22.05,26042.0,-4440.0,93540.0,-17.80,48.0,H-SC,6.42,164.0,-0.17,0.67,35.97,XR,ATH,FINANCE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.3,-0.59,109.15,107.91,160196.0,-877.0,146767.0,-18.93,60.0,M-SC,12.04,216.0,-0.01,1.04,9.43,OX40N,ATH,CABLES


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.30,-0.59,109.15,107.91,160196.0,-877.0,146767.0,-18.93,60.0,M-SC,12.04,216.0,-0.01,1.04,9.43,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-0.36,-15.87,18.89,0.02,27741.0,-27706.0,146853.0,123.77,43.0,M-SC,12.62,234.0,-1.00,1.05,27.74,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,0.02,-19.62,24.52,0.09,19862.0,-19773.0,81004.0,88.05,32.0,M-SC,2.27,253.0,-1.00,0.58,6.42,OX40N,NTT,DURABLES
72,TATAELXSI,9161.00,1.13,-21.10,67.34,32.03,70044.0,-27819.0,104016.0,-14.69,64.0,H-SC,9.32,148.0,-0.40,0.74,16.51,OX40N,NTT,IT
66,SIS,528.00,-0.36,-23.82,59.18,21.26,50057.0,-26448.0,84584.0,1995.85,52.0,H-SC,4.74,166.0,-0.53,0.60,14.28,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,2.91,11.62,54.33,72.27,102762.0,19696.0,189144.0,-4.26,77.0,M-LC,3.37,99.0,0.19,1.35,18.52,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,2.91,11.62,54.33,72.27,102762.0,19696.0,189144.0,-4.26,77.0,M-LC,3.37,99.0,0.19,1.35,18.52,XR,NTT,IT
25,FINCABLES,1641.55,-0.30,-0.59,109.15,107.91,160196.0,-877.0,146767.0,-18.93,60.0,M-SC,12.04,216.0,-0.01,1.04,9.43,OX40N,ATH,CABLES
6,ASIANTILES,137.00,1.29,-3.22,87.72,81.67,79638.0,-3023.0,90787.0,8172.22,72.0,L-SC,11.57,271.0,-0.04,0.65,75.81,XR,NTT,CERAMICS
38,INDIAMART,4810.62,-0.46,-5.19,118.03,106.72,138023.0,-6397.0,116939.0,-53.11,35.0,H-SC,3.98,139.0,-0.05,0.83,16.98,AR,ATH,MISC
85,ZYDUSLIFE,1286.17,0.36,-2.67,39.35,35.63,80263.0,-5602.0,203972.0,-16.30,43.0,H-MC,2.51,119.0,-0.07,1.45,13.55,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,0.62,-17.85,89.26,55.47,178391.0,-43430.0,199856.0,-27.18,32.0,X-MC,14.80,64.0,-0.24,1.42,2.40,X40N,ATH,FINANCE
8,AWL,485.00,-1.65,-23.79,101.41,53.50,233206.0,-71787.0,229964.0,-63.12,34.0,X-MC,4.82,58.0,-0.31,1.64,2.14,XY24,NTT,FMCG
11,BATAINDIA,2096.00,-0.23,-38.82,120.98,35.20,95243.0,-49944.0,78726.0,3.25,35.0,X-SC,16.76,92.0,-0.52,0.56,0.98,X40,NTT,FOOTWEAR
34,HONAUT,58357.33,-0.46,-17.41,73.53,43.32,98913.0,-28352.0,134520.0,-29.05,35.0,X-SC,5.43,90.0,-0.29,0.96,3.29,X40N,ATH,ELECTRICAL
22,DIXON,18931.72,-1.18,-12.69,44.40,26.07,75677.0,-24778.0,170443.0,-57.52,36.0,X-MC,11.31,56.0,-0.33,1.21,6.15,X40N,ATH,IT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,950.0,0.91,0.24,24.18,24.48,61042.0,594.0,252450.0,-80.87,50.0,X-SC,0.91,86.0,0.01,1.80,23.43,X40N,NTT,MISC
12,BERGEPAINT,680.0,0.93,-3.27,25.23,21.14,55485.0,-7424.0,219915.0,-10.11,45.0,X-MC,1.14,74.0,-0.13,1.56,23.96,XY24,NTT,PAINTS
20,DABUR,735.0,0.21,-3.22,48.48,43.70,116388.0,-7993.0,240075.0,-11.25,38.0,X-MC,2.06,72.0,-0.07,1.71,11.34,XY24,BTT,FMCG
43,ITC,452.0,0.54,-1.18,12.10,10.78,28736.0,-2833.0,237485.0,-41.15,50.0,X-LC,2.15,5.0,-0.10,1.69,4.00,X40,NTT,FMCG
78,UNITDSPR,1644.0,1.02,4.39,15.69,20.77,38348.0,10268.0,244412.0,-4.74,49.0,X-MC,2.16,62.0,0.27,1.74,11.61,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-0.23,-38.82,120.98,35.20,95243.0,-49944.0,78726.0,3.25,35.0,X-SC,16.76,92.0,-0.52,0.56,0.98,X40,NTT,FOOTWEAR
3,ACC,3906.00,1.12,-25.57,120.54,64.15,213488.0,-60841.0,177110.0,-56.31,39.0,X-SC,3.60,82.0,-0.28,1.26,1.12,XY24,BTT,CEMENT
19,COLPAL,3726.84,-0.02,-19.86,76.57,41.51,161616.0,-52295.0,211070.0,-4.54,41.0,X-MC,7.67,61.0,-0.32,1.50,1.12,XY25,ATH,FMCG
59,RELAXO,1176.00,0.27,-47.77,191.63,52.31,145277.0,-69349.0,75811.0,-44.44,42.0,X-SC,7.29,91.0,-0.48,0.54,1.48,X40N,NTT,FOOTWEAR
31,HAVELLS,2069.16,1.12,-9.28,46.03,32.48,136962.0,-30442.0,297549.0,-13.91,46.0,X-MC,6.35,67.0,-0.22,2.12,1.94,X40,ATH,ELECTRICAL


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,1.58,-28.47,99.71,42.84,46361.0,-18510.0,46496.0,-53.17,49.0,X-SC,34.53,83.0,-0.40,0.33,5.19,XY24,NTT,MISC
59,RELAXO,1176.00,0.27,-47.77,191.63,52.31,145277.0,-69349.0,75811.0,-44.44,42.0,X-SC,7.29,91.0,-0.48,0.54,1.48,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.23,-38.82,120.98,35.20,95243.0,-49944.0,78726.0,3.25,35.0,X-SC,16.76,92.0,-0.52,0.56,0.98,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,-0.47,-2.80,27.65,24.08,34119.0,-3555.0,123395.0,-10.62,41.0,X-SC,5.88,89.0,-0.10,0.88,16.27,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.46,-17.41,73.53,43.32,98913.0,-28352.0,134520.0,-29.05,35.0,X-SC,5.43,90.0,-0.29,0.96,3.29,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,0.83,-9.00,32.65,20.71,112374.0,-34059.0,344178.0,-21.68,72.0,X-LC,3.66,1.0,-0.30,2.45,15.00,X40,ATH,IT
41,INFY,1972.00,2.56,14.41,17.33,34.24,62039.0,45101.0,357989.0,-10.98,75.0,X-LC,5.32,2.0,0.73,2.55,23.89,X40,NTT,IT
75,TMPV,600.00,2.11,-26.19,66.62,22.98,111073.0,-59172.0,166726.0,-23.08,51.0,X-LC,6.54,3.0,-0.53,1.19,4.86,XY24,NTT,AUTO
81,VBL,671.64,3.06,-2.28,38.84,35.67,119873.0,-7210.0,308632.0,-13.89,59.0,X-LC,3.78,4.0,-0.06,2.20,11.26,X40N,ATH,FMCG
43,ITC,452.00,0.54,-1.18,12.10,10.78,28736.0,-2833.0,237485.0,-41.15,50.0,X-LC,2.15,5.0,-0.10,1.69,4.00,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,1.71,-36.57,109.51,32.89,53629.0,-28234.0,48972.0,-704.67,59.0,L-MC,6.63,259.0,-0.53,0.35,34.86,XR,NTT,BANKS
56,RAJESHEXPO,518.00,11.17,-56.11,128.01,0.07,77066.0,-76974.0,60203.0,2107.07,77.0,L-SC,3.07,268.0,-1.00,0.43,49.73,OX40N,NTT,JEWELLERY
6,ASIANTILES,137.00,1.29,-3.22,87.72,81.67,79638.0,-3023.0,90787.0,8172.22,72.0,L-SC,11.57,271.0,-0.04,0.65,75.81,XR,NTT,CERAMICS
50,MASFIN,398.61,-1.31,-4.53,27.84,22.05,26042.0,-4440.0,93540.0,-17.80,48.0,H-SC,6.42,164.0,-0.17,0.67,35.97,XR,ATH,FINANCE
13,BSOFT,831.70,2.99,-19.07,86.31,50.79,98248.0,-26821.0,113832.0,3.98,71.0,H-SC,7.22,151.0,-0.27,0.81,32.54,XR,ATH,IT


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RAJESHEXPO,518.00,11.17,-56.11,128.01,0.07,77066.0,-76974.0,60203.0,2107.07,77.0,L-SC,3.07,268.0,-1.00,0.43,49.73,OX40N,NTT,JEWELLERY
13,BSOFT,831.70,2.99,-19.07,86.31,50.79,98248.0,-26821.0,113832.0,3.98,71.0,H-SC,7.22,151.0,-0.27,0.81,32.54,XR,ATH,IT
61,ROUTE,2227.21,-0.23,-45.48,210.09,69.05,152406.0,-60523.0,72543.0,-55.87,56.0,H-SC,21.34,140.0,-0.40,0.52,11.88,SR,ATH,IT
6,ASIANTILES,137.00,1.29,-3.22,87.72,81.67,79638.0,-3023.0,90787.0,8172.22,72.0,L-SC,11.57,271.0,-0.04,0.65,75.81,XR,NTT,CERAMICS
84,WIPRO,420.00,2.91,11.62,54.33,72.27,102762.0,19696.0,189144.0,-4.26,77.0,M-LC,3.37,99.0,0.19,1.35,18.52,XR,NTT,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.44
1,25,44.86
2,50,76.59


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.91
MC    30.76
LC    24.34
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.74
X40      23.08
X40N     13.01
XR        9.87
XY25      9.05
AR        8.90
OX40N     7.59
SR        1.01
MH        0.76
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.26
H-SC    23.65
X-LC    20.87
M-SC    11.66
X-SC     8.05
H-MC     4.80
L-SC     1.55
M-LC     1.35
M-MC     1.35
H-LC     1.11
L-LC     1.01
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.29,-7.43,41.97
IT,13.23,-13.13,70.01
FINANCE,9.45,-19.93,72.10
MISC,7.28,-27.08,79.07
ELECTRICAL,5.96,-12.10,53.16
PAINTS,5.79,-13.46,30.32
INSURANCE,4.74,-1.78,36.62
PHARMA,4.09,-2.26,34.45
AUTO,2.86,-30.45,75.68


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3163902.0,21
AR,1319845.0,10
XR,1277043.0,13
X40,981425.0,14
X40N,857811.0,9
OX40N,724992.0,10
XY25,368609.0,6
SR,278420.0,2
MH,77731.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3653939.0,25
M-SC,1420306.0,15
X-MC,1417833.0,16
X-LC,793853.0,11
X-SC,789415.0,8
H-MC,404385.0,3
L-SC,243926.0,3
M-LC,102762.0,1
H-LC,77267.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1278131.0      6
           AR         906426.0      5
M-SC       XY24       807886.0      6
H-SC       XR         780334.0      7
X-MC       X40        489219.0      7
           XY24       405079.0      3
           X40N       340168.0      4
H-SC       OX40N      332897.0      4
M-SC       OX40N      315029.0      5
X-SC       X40N       305232.0      3
X-LC       X40        301991.0      5
X-SC       XY24       293968.0      3
H-SC       SR         278420.0      2
X-LC       X40N       212411.0      2
H-MC       AR         212219.0      2
           XY24       192166.0      1
X-SC       X40        190215.0      2
X-LC       XY24       186672.0      2
X-MC       XY25       183367.0      2
M-SC       XR         173458.0      2
L-SC       XR         166860.0      2
M-SC       AR         123933.0      2
M-LC       XR         102762.0      1
X-LC       XY25        92779.0      2
H-SC       MH          77731.0      1
H-LC       AR          77267.0      1
L-SC       OX40N       77066.0      1
M-MC       XY25        55559.0      1
L-MC       XR          53629.0      1
L-LC       XY25        36904.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 40.0 seconds
